# Import dependencies


In [46]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [58]:
# Example list of words
words =words = [
    "apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", 
    "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", 
    "ugli", "watermelon", "apricot", "blackberry", "blueberry", "cantaloupe", "coconut", "cranberry", 
    "currant", "dragonfruit", "gooseberry", "grapefruit", "guava", "jackfruit", "kumquat", "lime", 
    "lychee", "mulberry", "olive", "passionfruit", "peach", "pear", "persimmon", "pineapple", "plum", 
    "pomegranate", "starfruit", "tomato", "avocado", "blackcurrant", "boysenberry", "cucumber", 
    "durian", "pluot", "rhubarb", "apple", "orange", "peach", "red", "blue", "green", "yellow", 
    "pink", "purple", "brown", "black", "white", "gray", "teal", "cyan", "magenta", "violet", 
    "turquoise", "maroon", "beige", "gold", "silver", "indigo", "lavender", "peachpuff", "crimson", 
    "salmon", "coral", "tomato", "limegreen", "darkgreen", "sienna", "fuchsia", "lemonchiffon", 
    "peru", "chocolate", "lightblue", "navy", "khaki", "plum", "tan", "azure", "orchid", "mint"
]



# Create character mappings
all_chars = sorted(set(''.join(words)))
char2idx = {char: idx for idx, char in enumerate(all_chars)}
idx2char = {idx: char for char, idx in char2idx.items()}
print(f"Character mappings: {char2idx}")
print(f"Character mappings: {idx2char}")

Character mappings: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'y': 23, 'z': 24}
Character mappings: {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'y', 24: 'z'}


In [59]:
def create_character_pairs(words):
    x_data = []
    y_data = []
    
    for word in words:
        for i in range(len(word) - 1):
            x = word[:i+1]  # Input sequence (up to the current character)
            y = word[i+1]   # Output character (the next character)
            
            # Convert characters to indices
            x_indices = [char2idx[char] for char in x]
            y_index = char2idx[y]
            
            x_data.append(x_indices)
            y_data.append(y_index)
    
    return x_data, y_data

In [60]:
x_data, y_data = create_character_pairs(words)
print(x_data)
max_length = max(len(seq) for seq in x_data)
x_data_padded = tf.keras.preprocessing.sequence.pad_sequences(x_data, maxlen=max_length, padding='post')
y_data = np.array(y_data).reshape(-1, 1)
print(x_data_padded)
print(y_data)

[[0], [0, 15], [0, 15, 15], [0, 15, 15, 11], [1], [1, 0], [1, 0, 13], [1, 0, 13, 0], [1, 0, 13, 0, 13], [2], [2, 7], [2, 7, 4], [2, 7, 4, 17], [2, 7, 4, 17, 17], [3], [3, 0], [3, 0, 19], [4], [4, 11], [4, 11, 3], [4, 11, 3, 4], [4, 11, 3, 4, 17], [4, 11, 3, 4, 17, 1], [4, 11, 3, 4, 17, 1, 4], [4, 11, 3, 4, 17, 1, 4, 17], [4, 11, 3, 4, 17, 1, 4, 17, 17], [5], [5, 8], [6], [6, 17], [6, 17, 0], [6, 17, 0, 15], [7], [7, 14], [7, 14, 13], [7, 14, 13, 4], [7, 14, 13, 4, 23], [7, 14, 13, 4, 23, 3], [7, 14, 13, 4, 23, 3, 4], [10], [10, 8], [10, 8, 22], [11], [11, 4], [11, 4, 12], [11, 4, 12, 14], [12], [12, 0], [12, 0, 13], [12, 0, 13, 6], [13], [13, 4], [13, 4, 2], [13, 4, 2, 19], [13, 4, 2, 19, 0], [13, 4, 2, 19, 0, 17], [13, 4, 2, 19, 0, 17, 8], [13, 4, 2, 19, 0, 17, 8, 13], [14], [14, 17], [14, 17, 0], [14, 17, 0, 13], [14, 17, 0, 13, 6], [15], [15, 0], [15, 0, 15], [15, 0, 15, 0], [15, 0, 15, 0, 23], [16], [16, 20], [16, 20, 8], [16, 20, 8, 13], [16, 20, 8, 13, 2], [17], [17, 0], [17, 0, 

In [61]:
#build model
model = Sequential()
model.add(SimpleRNN(32, input_shape=(None, 1)))
model.add(Dense(len(all_chars), activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_data_padded, y_data, epochs=1000, verbose=1)

Epoch 1/1000


/opt/conda/envs/quant/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0362 - loss: 3.2434   
Epoch 2/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1035 - loss: 3.1248 
Epoch 3/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1452 - loss: 3.0096 
Epoch 4/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1656 - loss: 2.9303 
Epoch 5/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1778 - loss: 2.8753 
Epoch 6/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1684 - loss: 2.8975 
Epoch 7/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2053 - loss: 2.7915 
Epoch 8/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1707 - loss: 2.8146 
Epoch 9/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1707 - loss: 2.8104 
Epoch 10/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1631 - loss: 2.7688 
Epoch 11/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1715 - loss: 2.7681 
Epoch 12/1000
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [47]:
# Testing the model
def predict_next_char(input_word):
    input_indices = [char2idx[char] for char in input_word]
    input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_indices], maxlen=max_length, padding='post')
    
    predictions = model.predict(input_padded)
    predicted_char_index = np.argmax(predictions[0])
    
    return idx2char[predicted_char_index]

In [66]:
# Examples list for predictions
examples = [
    ["appl",'e' ] # "apple" without the last char
    , ["orang",'e' ] # "orange" without the last char
    , ["peac",'h' ] # "peach" without the last char
    , ["re",'d' ] # "red" without the last char
    , ["blu",'e' ] # "blue" without the last char
    , ["gree",'n' ] # "green" without the last char
    , ["yell",'o' ] # "yellow" without the last char
    , ["purp",'l' ] # "purple" without the last char
    , ["brow",'n' ] # "brown" without the last char
    , ["blac",'k' ] # "black" without the last char
    , ["whit",'e' ] # "white" without the last char
    , ["gra",'y' ] # "gray" without the last char
    , ["tea",'l' ] # "teal" without the last char
    , ["cy",'a' ] # "cyan" without the last char
    , ["magen",'t' ] # "magenta" without the last char
    , ["viole",'t' ] # "violet" without the last char
]

predicted_char=[]
for input_word in examples:
    predicted_char.append(predict_next_char(input_word[0]))

for i in range(len(examples)):
    print(f"Next character prediction for '{examples[i][0]}' is '{predicted_char[i]}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Next character prediction for 'appl' is 'e'
Next character prediction for 'orang' is 'e'
Next character prediction for 'peac' is 'h'
Next character prediction for 're' is 'd'
Next character prediction for 'blu' is 'e'
Next character prediction for 'gree' is 'n'
Next character prediction for 'yell' is 'o'
Next character prediction for 'purp' is 'u'
Next character prediction for 'brow' is 'n'
Next character prediction for 'blac' is 'k'
Next character prediction for 